## Purpose

This notebook is to test whether the `neuro_op` module's nodes can successfully infer `world_dist` if only this distribution's information pieces $x_{world}$ are received (i.e., with rates h>0, r=0).

For this, the model is run with nodes receiving different amount of information (i.e., different `t_max`, `h`). Then, each node's posterior predictive distribution (*PPD*), equalling its forecast of future incoming information, is obtained via 
1. sampling of model parameters $\theta$ proportional to its posterior $p(\theta | x_{world})$;
2. using these sampled model parameters to generate data proportional to the model likelihood $p(x_{PPD}|\theta_{sampled})$

PPDs thereby fully conserves uncertainty by paying respect to both posterior and likelihood stochasticity.

We then use the PPDs to quantify the nodes' modelling accuracy by computing the Kullback-Leibler divergence and average MLE distances between node PPDs and data generated by `world_dist`.

In [1]:
import copy  # deep-copying of input dictionary (which includes mutable objects)
import gc  # explicit garbace collection calling after each run
import neuro_op as nop  # project's main module
import matplotlib.pyplot as plt  # because Figures >> Text

# import networkx as nx          # network initialization
import numpy as np  # ...of course we need numpy
import pickle  # output export/import
import scipy.stats as st  # ...of course we need scipy
import time  # runtime measuring

#### Figure 1, 2

Varying parameters:
- $t_{max} = 1eX, X\in\{2,3,4,5,6,7^?\}$
- $N_{agents} = 1eX, X\in\{1,2,3,4,5\}$

Figure 1:
- histogram $\mu_i$ (for now MLEs)

Figure 2:
- $3d^?$ plot w. $(x,y,z) = (t_{max}, N_{agents}, t_{sim})$

In [47]:
def model_scan(dict_list):
    """
    Call 'run_model' with 'input_standard' adapted at specified dictionary entries.

    Serially run multiple model parameter sets, safe output to pickle file, garbage collect memory.
    """

    for dic_tmp in dict_list:
        input = copy.deepcopy(nop.input_standard)
        for key, value in dic_tmp.items():
            input[key] = value
        print("Current adaptions:\t", dic_tmp.items())
        t0 = time.time()
        output = dict(nop.run_model(**input))
        t1 = time.time()
        output["t_exec"] = t1 - t0
        print("For adaptions\t", dic_tmp.items(), " :\n\t t_exec = ", (t1 - t0))
        filename = (
            "out"
            + str(dic_tmp)
            + time.strftime("--%Y-%m-%d--%H-%M--", time.localtime(t0))
            + "--export.pkl"
        )
        with open(filename, "wb") as f:
            pickle.dump(output, f)
        del output
        gc.collect()

Current adaptions:	 dict_items([('N_nodes', 10)])
For adaptions
 dict_items([('N_nodes', 10)])  :
t_exec =  0.6124289035797119


In [75]:
params = []
for N in [1, 2, 3, 4]:
    for t in [1, 2, 3, 4]:
        params.append(dict(N_nodes=10**N, t_max=10**t))

for t in [5, 6]:
    params.append(dict(t_max=10**t))

for r, h in [(0.1, 5), (5, 0.1), (5, 5)]:
    for N in [1, 2, 3, 4]:
        params.append(dict(N_nodes=10**N, r=r, h=h))

for N in [5]:
    params.append(dict(N_nodes=10**N))

params

[{'N_nodes': 10, 't_max': 10},
 {'N_nodes': 10, 't_max': 100},
 {'N_nodes': 10, 't_max': 1000},
 {'N_nodes': 10, 't_max': 10000},
 {'N_nodes': 100, 't_max': 10},
 {'N_nodes': 100, 't_max': 100},
 {'N_nodes': 100, 't_max': 1000},
 {'N_nodes': 100, 't_max': 10000},
 {'N_nodes': 1000, 't_max': 10},
 {'N_nodes': 1000, 't_max': 100},
 {'N_nodes': 1000, 't_max': 1000},
 {'N_nodes': 1000, 't_max': 10000},
 {'N_nodes': 10000, 't_max': 10},
 {'N_nodes': 10000, 't_max': 100},
 {'N_nodes': 10000, 't_max': 1000},
 {'N_nodes': 10000, 't_max': 10000},
 {'t_max': 100000},
 {'t_max': 1000000},
 {'N_nodes': 10, 'r': 0.1, 'h': 5},
 {'N_nodes': 100, 'r': 0.1, 'h': 5},
 {'N_nodes': 1000, 'r': 0.1, 'h': 5},
 {'N_nodes': 10000, 'r': 0.1, 'h': 5},
 {'N_nodes': 10, 'r': 5, 'h': 0.1},
 {'N_nodes': 100, 'r': 5, 'h': 0.1},
 {'N_nodes': 1000, 'r': 5, 'h': 0.1},
 {'N_nodes': 10000, 'r': 5, 'h': 0.1},
 {'N_nodes': 10, 'r': 5, 'h': 5},
 {'N_nodes': 100, 'r': 5, 'h': 5},
 {'N_nodes': 1000, 'r': 5, 'h': 5},
 {'N_nodes

In [ ]:
model_scan(params)